In [0]:
%matplotlib inline 
import numpy as np 
import pandas as pd 
import geopandas as gpd 
from folium.plugins import MarkerCluster
from shapely.geometry import Point
import json
import folium

In [0]:
gdf = gpd.read_file("./shp/h27ka13112.shp")

In [17]:
gdf.head(2)

,KEY_CODE,PREF,CITY,S_AREA,PREF_NAME,CITY_NAME,S_NAME,KIGO_E,HCODE,AREA,...,KIGO_I,MOJI,KBSUM,JINKO,SETAI,X_CODE,Y_CODE,KCODE1,geom_wkt,geometry
0,13112001001,13,112,001001,東京都,世田谷区,池尻１丁目,None,8101,202181.278,...,None,池尻１丁目,15,1635,685,139.68211,35.64403,0010-01,"MultiPolygon (((139.68514746 35.64135584, 139....","POLYGON ((139.6851474624769 35.64135583850746,..."
1,13112001002,13,112,001002,東京都,世田谷区,池尻２丁目,None,8101,188295.440,...,None,池尻２丁目,55,4766,2888,139.68165,35.64772,0010-02,"MultiPolygon (((139.681702 35.64583059, 139.68...","POLYGON ((139.6817019995719 35.6458305892883, ..."


In [0]:
df = pd.read_csv("./peseudo_rent_data_utf8.csv")

In [22]:
df.head(2)

,No,都道府県,市区町村,町丁目,築年,築月,構造,マンション最上階数,間取別平均平米単価①,間取別平均平米単価②,...,間取別平均平米単価④,間取別平均平米単価⑤,物件緯度,物件経度,最寄駅,最寄駅緯度,最寄駅経度,始点ノード,終点ノード,最寄駅までの距離
0,1,東京都,世田谷区,喜多見７丁目,1988,3,ＲＣ,2,2812.2742,2311.9321,...,2145.7930,2634.3530,35.631294,139.594005,喜多見,35.636745,139.587513,1.395870e+14,1.395930e+14,1096.93
1,2,東京都,世田谷区,喜多見７丁目,2005,3,鉄骨造,3,3006.8074,2633.6113,...,2573.4016,2733.7278,35.632051,139.592723,喜多見,35.636745,139.587513,1.395870e+14,1.395920e+14,930.57


In [0]:
chome_df = pd.DataFrame(df.groupby(['都道府県', '市区町村', '町丁目'])['間取別平均平米単価①', '間取別平均平米単価②', '間取別平均平米単価③', '間取別平均平米単価④', '間取別平均平米単価⑤'].mean())


In [0]:
chome_df = chome_df.reset_index()


In [30]:
chome_df.head(2)

,都道府県,市区町村,町丁目,間取別平均平米単価①,間取別平均平米単価②,間取別平均平米単価③,間取別平均平米単価④,間取別平均平米単価⑤
0,東京都,世田谷区,三宿１丁目,3428.933440,3110.774560,2909.462413,2771.685420,2634.315673
1,東京都,世田谷区,三宿２丁目,3455.780079,3140.654237,2919.330146,2755.093342,2680.324733


In [31]:
len(chome_df)

270

In [32]:
len(gdf)

277

In [0]:
gdf = gdf.rename(columns={'MOJI': '町丁目'})

In [0]:
_gdf = pd.merge(gdf, chome_df, on = '町丁目', how = 'inner')


In [35]:
len(_gdf)

270

In [36]:
_gdf.head(2)

,KEY_CODE,PREF,CITY,S_AREA,PREF_NAME,CITY_NAME,S_NAME,KIGO_E,HCODE,AREA,...,KCODE1,geom_wkt,geometry,都道府県,市区町村,間取別平均平米単価①,間取別平均平米単価②,間取別平均平米単価③,間取別平均平米単価④,間取別平均平米単価⑤
0,13112001001,13,112,001001,東京都,世田谷区,池尻１丁目,None,8101,202181.278,...,0010-01,"MultiPolygon (((139.68514746 35.64135584, 139....","POLYGON ((139.6851474624769 35.64135583850746,...",東京都,世田谷区,3289.278720,2943.472480,2847.79064,2824.34358,2582.165060
1,13112001002,13,112,001002,東京都,世田谷区,池尻２丁目,None,8101,188295.440,...,0010-02,"MultiPolygon (((139.681702 35.64583059, 139.68...","POLYGON ((139.6817019995719 35.6458305892883, ...",東京都,世田谷区,3465.723207,3101.954867,2972.29778,2889.19292,2630.925493


In [57]:
#bins = list(_gdf['間取別平均平米単価①'].quantile([0, 0.25, 0.5, 0.75, 1]))

m = folium.Map(location=[35.643554, 139.671146],
                   tiles='https://cyberjapandata.gsi.go.jp/xyz/seamlessphoto/{z}/{x}/{y}.jpg',
                   attr='国土地理院 空中写真',
                   zoom_start=12)

folium.Choropleth(
    geo_data=_gdf,
    name='choropleth',
    data=_gdf,
    columns=['町丁目', '間取別平均平米単価①'],
    key_on='feature.properties.町丁目',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='間取別平均平米単価①'
).add_to(m)

folium.LayerControl().add_to(m)

m

In [46]:
_gdf.dtypes

KEY_CODE       object
PREF           object
CITY           object
S_AREA         object
PREF_NAME      object
CITY_NAME      object
S_NAME         object
KIGO_E         object
HCODE           int64
AREA          float64
PERIMETER     float64
H27KAxx_        int64
H27KAxx_ID      int64
KEN            object
KEN_NAME       object
SITYO_NAME     object
GST_NAME       object
CSS_NAME       object
KIHON1         object
DUMMY1         object
KIHON2         object
KEYCODE1       object
KEYCODE2       object
AREA_MAX_F     object
KIGO_D         object
N_KEN          object
N_CITY         object
KIGO_I         object
町丁目            object
KBSUM           int64
JINKO           int64
SETAI           int64
X_CODE        float64
Y_CODE        float64
KCODE1         object
geom_wkt       object
geometry       object
都道府県           object
市区町村           object
間取別平均平米単価①    float64
間取別平均平米単価②    float64
間取別平均平米単価③    float64
間取別平均平米単価④    float64
間取別平均平米単価⑤    float64
dtype: object